In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import source.spectra_0308 as sp
import seaborn as sns
import pandas as pd
from math import ceil
from os.path import exists

In [2]:
TIME_UNIT = 0.016*2**5
NCH = 32

def trigger_sample(start_conv):
    mid_value = np.take(start_conv, start_conv.size // 2)
    return np.argwhere(start_conv[1:] - start_conv[:-1] > mid_value/5)[0][0]

def get_mux_intervals(start_conv,
                      tunit_us = TIME_UNIT,
                      mux_time_us = 10.0,
                      offset_from_start_us = 0.,
                      offset_from_stop_us= 0.):
    
    ts = trigger_sample(start_conv)
    mux_intervals = [
        [ceil(ts + offset_from_start_us / tunit_us + i * mux_time_us / tunit_us),
         ceil(ts - offset_from_stop_us / tunit_us + (i + 1) * mux_time_us / tunit_us)]
        for i in range(NCH)
                    ]
    return mux_intervals

def compute_duration(tmux, mux_intervals, weird_id):
    return TIME_UNIT*np.sum(tmux[slice(*mux_intervals[weird_id])])/max(tmux[slice(*mux_intervals[weird_id])])

In [3]:
def ok(file):
    return 'red' not in file and 'bkg' not in file and 'QUADD' not in file and 'Am241' not in file

def get_fileid(file):
    return '_'.join(file[:-5].split('/')[-1].split('_')[:2])

def get_radsource(file):
    return file[:-5].split('_')[-1]

def get_detmodel(file):
    return file[:-5].split('_')[2]

def get_quadrant(file):
    return file[:-5].split('_')[4][-1]

In [4]:
files_fm1 = ["D:/IAPSdata/FM1/20220304/" + f for f in os.listdir("D:/IAPSdata/FM1/20220304/") if ok(f)][1:] + ["D:/IAPSdata/FM1/20220301/" + f for f in os.listdir("D:/IAPSdata/FM1/20220301/") if ok(f)]

In [5]:
files_pfm =  ["D:/IAPSdata/PFM/20220310/" + f for f in os.listdir("D:/IAPSdata/PFM/20220310/") if 'red' not in f and 'Am' not in f] + ["D:/IAPSdata/PFM/20220307/" + f for f in os.listdir("D:/IAPSdata/PFM/20220307/") if 'red' not in f]  +  ["D:/IAPSdata/PFM/20220308/" + f for f in os.listdir("D:/IAPSdata/PFM/20220308/") if 'red' not in f and 'Am' not in f and 'Fe' not in f and '07' not in f]

In [6]:
for ff in files_pfm + files_fm1:
    if exists(ff):
        pass
    else:
        print(ff, "not exist")
        break

In [7]:
weird_trigs = []

for filepath in files_fm1 + files_pfm:
    el_1 = sp.event_list_from_wfs(filepath, offset_from_start_us=2.5, offset_from_stop_us=2.5)
    el_0 = sp.event_list_from_wfs(filepath)
    decimation, file_ts = sp.get_meta(filepath)


    wfs, txs = sp.get_waveforms(filepath, wf_ch = 'CH00', tmux_ch = 'CH12')
    _, scs = sp.get_waveforms(filepath, wf_ch = 'CH00', tmux_ch = 'CH08')

    ntrig = len(el_0)
    tot = 0

    for i in range(ntrig):
        if not np.all(np.isnan(el_1[i]) == np.isnan(el_0[i])):
            tot += 1
            times = np.arange(len(wfs[i]))*TIME_UNIT
            mux_intervals = get_mux_intervals(scs[i])

            file_id = get_fileid(filepath)
            index = i
            weird_id = np.argwhere(np.logical_not((np.isnan(el_1[i]))) & (np.isnan(el_0[i])))[0][0]
            detmod = get_detmodel(filepath)
            quad = get_quadrant(filepath)
            radsrc = get_radsource(filepath)
            duration = compute_duration(txs[i], mux_intervals, weird_id)
            
            weird_trigs.append((file_id, detmod, quad, radsrc, index, weird_id, duration))

            with sns.plotting_context("talk"):
                plt.figure(figsize = (15,5))
                plt.plot(times, wfs[i])
                plt.plot(times, txs[i])
                plt.plot(times, scs[i])
                plt.axvspan(*times[mux_intervals[weird_id]], color = 'red', alpha = 0.1)
                plt.axvline(times[trigger_sample(scs[i])], linestyle = 'dotted')
                plt.xlabel("time [us]")
                plt.ylabel("adc")
                plt.savefig("out/plots/wt_" + '_'.join([detmod,file_id, "QUAD" + quad, radsrc,  str(i)]),bbox_inches='tight')
                plt.close()
    print("found",tot,"weird tr in",filepath.split('/')[-1])

found 62 weird tr in 20220304_02_FM1_T22deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Fe55.fits
found 74 weird tr in 20220304_03_FM1_T22deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Fe55.fits
found 106 weird tr in 20220304_04_FM1_T23deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Cd109.fits
found 87 weird tr in 20220304_06_FM1_T23deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Cd109.fits
found 87 weird tr in 20220304_07_FM1_T23deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Cs137.fits
found 80 weird tr in 20220304_08_FM1_T23deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Cs137.fits
found 74 weird tr in 20220304_09_FM1_T23deg_QUADC_THR110_finethr_DAC120_SingleCH_HV123_no_CH02_03_05_06_07_08_09_10_13_17_29_31_Cs137.fits
found 129 weird tr in 202203

In [8]:
weird_trigs_df = pd.DataFrame(weird_trigs, columns = ['file_id', 'det_model', 'quadrant', 'source', 'index', 'ch_id', 'duration'])

In [11]:
weird_trigs_df.to_csv("out/weird_trigs.csv", index = False)

In [18]:
obs = []

for filepath in files_fm1 + files_pfm:
    wfs, txs = sp.get_waveforms(filepath, wf_ch = 'CH00', tmux_ch = 'CH12')
    _, scs = sp.get_waveforms(filepath, wf_ch = 'CH00', tmux_ch = 'CH08')

    file_id = get_fileid(filepath)
    detmod = get_detmodel(filepath)
    quad = get_quadrant(filepath)
    radsrc = get_radsource(filepath)
    acquisitions = len(wfs)
    ws = weird_trigs_df[weird_trigs_df['file_id'] == file_id].shape[0]
    
    obs.append((file_id, detmod, quad, radsrc, acquisitions, ws))

In [19]:
obs_df = pd.DataFrame(obs, columns = ['file_id', 'det_model', 'quadrant', 'source', 'tot_acq', 'weird_acq'])
obs_df.to_csv("out/obs.csv", index= False)